**Создать вычислительный граф функции**  
Вычислить значение функции для некоторых значений переменных  
  
***Функция 6***: 2x-3y+3x+2y

In [4]:
class Operation():
    
    def __init__(self, input_nodes = []):
        self.input_nodes = input_nodes
        self.output_nodes = []
        
        for node in input_nodes:
            node.output_nodes.append(self)
        
        _default_graph.operations.append(self)
  
    def compute(self):
        
        pass

In [5]:
class add(Operation):
    
    def __init__(self, x, y):
        super().__init__([x, y])

    def compute(self, x_var, y_var):
        self.inputs = [x_var, y_var]
        return x_var + y_var

In [6]:
class multiply(Operation):
     
    def __init__(self, a, b):
        
        super().__init__([a, b])
    
    def compute(self, a_var, b_var):
         
        self.inputs = [a_var, b_var]
        return a_var * b_var

In [8]:
class Placeholder():
    def __init__(self):
        
        self.output_nodes = []
        
        _default_graph.placeholders.append(self)

In [9]:
class Variable():
    def __init__(self, initial_value = None):
        
        self.value = initial_value
        self.output_nodes = []
        
        _default_graph.variables.append(self)

In [10]:
class Graph():
    def __init__(self):
        self.operations = []
        self.placeholders = []
        self.variables = []
        
    def set_as_default(self):
        global _default_graph
        _default_graph = self

In [24]:
def traverse_postorder(operation):
    nodes_postorder = []
    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)

    recurse(operation)
    return nodes_postorder

In [25]:
class Session:
    
    def run(self, operation, feed_dict = {}):
        nodes_postorder = traverse_postorder(operation)
        
        for node in nodes_postorder:
            if type(node) == Placeholder:
                
                node.output = feed_dict[node]
                
            elif type(node) == Variable:
                
                node.output = node.value
                
            else:                
                
                node.inputs = [input_node.output for input_node in node.input_nodes]
                 
                node.output = node.compute(*node.inputs)
                
            if type(node.output) == list:
                node.output = np.array(node.output)
        
        return operation.output

In [26]:
g = Graph()
g.set_as_default()

In [27]:
x = Placeholder()
y = Placeholder()
a = Variable(2)
b = Variable(3)
c = Variable(-3)

In [28]:
#2x-3y+3x+2y
d = multiply(x,a)
e = multiply(y,c)
f = multiply(x,b)
g = multiply(y,a)

h = add(d,e)
i = add(h,f)
j = add(i,g)

In [29]:
sess = Session()

In [30]:
test_list = [[0,0], [1,0], [0,1], [3,2], [3,-2]]


In [31]:
for ind, test in enumerate(test_list):
    print('Тест' + str(ind+1)+': '+ str(test[0])+str(test[1]))
    print(sess.run(operation=j,feed_dict={x:test[0], y:test[1]}))

Тест1: 00
0
Тест2: 10
5
Тест3: 01
-1
Тест4: 32
13
Тест5: 3-2
17
